<a href="https://colab.research.google.com/github/quasar-in-the-mist/Numerical-Relativity/blob/main/World_Bank_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install world_bank_data --upgrade

In [2]:
import pandas as pd
import world_bank_data as wb
pd.set_option('display.max_rows', 6)

In [3]:
wb.get_topics()

,value,sourceNote
id,,
1,Agriculture & Rural Development,For the 70 percent of the world's poor who liv...
2,Aid Effectiveness,Aid effectiveness is the impact that aid has i...
3,Economy & Growth,Economic growth is central to economic develop...
...,...,...
19,Climate Change,Climate change is expected to hit developing c...
20,External Debt,Debt statistics provide a detailed picture of ...
21,Trade,Trade is a key means to fight poverty and achi...


In [4]:
wb.get_sources()

,lastupdated,name,code,description,url,dataavailability,metadataavailability,concepts
id,,,,,,,,
1,2021-08-18,Doing Business,DBS,,,Y,Y,3
2,2025-04-15,World Development Indicators,WDI,,,Y,Y,3
3,2024-11-05,Worldwide Governance Indicators,WGI,,,Y,Y,3
...,...,...,...,...,...,...,...,...
90,2024-08-04,ICP 2021,IC2,,,Y,Y,4
91,2025-01-31,PEFA_CRPFM,CRP,,,Y,Y,4
92,2025-03-28,Disability Data Hub (DDH),DDH,,,Y,Y,3


In [5]:
wb.get_countries()

,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
id,,,,,,,,,
ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.5167
AFE,ZH,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates,,NaN,NaN
AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
...,...,...,...,...,...,...,...,...,...
ZAF,ZA,South Africa,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Upper middle income,IBRD,Pretoria,28.1871,-25.7460
ZMB,ZM,Zambia,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,IDA,Lusaka,28.2937,-15.3982
ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312


In [7]:
wb.get_regions
wb.get_incomelevels
wb.get_lendingtypes

<function world_bank_data.others.get_lendingtypes(lendingtype=None, language=None, **params)>

In [2]:
pip install wbdata

In [9]:
# =====================

# Download “Agriculture, value added (current US$)”
# # (indicator code: NV.AGR.TOTL.CD) from the World Bank,
# # persist it with SQLAlchemy, and run a sample query.


# import os
import wbdata                            # -> pip install wbdata
# import pandas as pd                     # -> pip install pandas
# from sqlalchemy import (
#     create_engine, MetaData, Table, Column, Integer,
#     Float, String, select, func, desc
# )
# from sqlalchemy.orm import sessionmaker

# --------------------------------------------------------------------------- #
# 1. Download data from the World Bank API
# --------------------------------------------------------------------------- #

INDICATOR = "NV.AGR.TOTL.CD"            # Agriculture, value added (current US$)
YEAR_RANGE = (1960, 2025)               # pull the full available range

print("Fetching data from the World Bank …")
df = (
    wbdata.get_dataframe({INDICATOR: "agri_gdp_usd"})
        .reset_index()                  # brings country & date out of the index
        .rename(columns={"country": "country_name",
                         "date": "year"})
        .dropna(subset=["agri_gdp_usd"])
)



df


Fetching data from the World Bank …


,country_name,year,agri_gdp_usd
1,Africa Eastern and Southern,2023,1.772532e+11
2,Africa Eastern and Southern,2022,1.649018e+11
3,Africa Eastern and Southern,2021,1.455380e+11
4,Africa Eastern and Southern,2020,1.366884e+11
5,Africa Eastern and Southern,2019,1.275033e+11
...,...,...,...
17280,Zimbabwe,1969,3.415027e+08
17281,Zimbabwe,1968,2.501005e+08
17282,Zimbabwe,1967,3.053436e+08
17283,Zimbabwe,1966,2.707916e+08


In [ ]:

# # convert the year to an integer (comes back as Timestamp when convert_date=True)
# df["year"] = df["year"].dt.year.astype(int)

# # Enrich with ISO-3 codes (handy for joins/filters later)
# COUNTRY_DICT = {c['name']: c['id'] for c in wbdata.get_country()}
# df["country_code"] = df["country_name"].map(COUNTRY_DICT)

# print(f"{len(df):,} rows downloaded.")

# # --------------------------------------------------------------------------- #
# # 2. Persist the data in a database via SQLAlchemy
# # --------------------------------------------------------------------------- #

# # Use SQLite unless the user points SQLAlchemy at something else
# DB_URI = os.getenv("WB_SQLALCHEMY_URI", "sqlite:///worldbank.db")
# engine = create_engine(DB_URI, echo=False)

# metadata = MetaData()

# agri_gdp_table = Table(
#     "agri_gdp",
#     metadata,
#     Column("id", Integer, primary_key=True),
#     Column("country_code", String(3), index=True),
#     Column("country_name", String(128)),
#     Column("year", Integer, index=True),
#     Column("agri_gdp_usd", Float)
# )

# metadata.create_all(engine)  # create table if it does not yet exist

# # Replace existing data for a clean demo
# with engine.begin() as conn:
#     conn.execute(agri_gdp_table.delete())      # wipe previous run (optional)
#     df.to_sql("agri_gdp", conn, if_exists="append", index=False)

# print("Data stored successfully in the database.")

# # --------------------------------------------------------------------------- #
# # 3. Query the data with SQLAlchemy
# # --------------------------------------------------------------------------- #


In [ ]:
# Session = sessionmaker(bind=engine)
# session = Session()

# # Sub-query: latest year per country
# latest_year_sub = (
#     select(
#         agri_gdp_table.c.country_code,
#         func.max(agri_gdp_table.c.year).label("latest_year")
#     )
#     .group_by(agri_gdp_table.c.country_code)
# ).subquery()

# # Join with the main table to fetch the value corresponding to that latest_year
# stmt = (
#     select(
#         agri_gdp_table.c.country_code,
#         agri_gdp_table.c.country_name,
#         agri_gdp_table.c.year,
#         agri_gdp_table.c.agri_gdp_usd
#     )
#     .join(latest_year_sub,
#           (agri_gdp_table.c.country_code == latest_year_sub.c.country_code) &
#           (agri_gdp_table.c.year == latest_year_sub.c.latest_year)
#     )
#     .order_by(desc(agri_gdp_table.c.agri_gdp_usd))
# )

# print("\nTop 10 countries by Agriculture GDP (US$) in their most recent year:")
# for row in session.execute(stmt.limit(10)):
#     print(f"{row.country_name:30} {row.agri_gdp_usd:,.0f}  (year {row.year})")

